## Section 1: Data Scraping

First, we will need to collect all relevant data.

Data will be from all Arsenal premier league matches from the 2020/21 season to the 2023/24 season.

For each match, we will collect the opponent's in-game data, and will later use this to categorise their 'style of play' based on thresholds for each style metric.

Categories that we will use include shooting, passing, pass types, goal and shot creation, defensive actions, possession, and misc (cards, fowls, etc).

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np

In [3]:
def get_table(url, table_id):
  try:
    response = requests.get(url)
  except:
    print("Can't find url")
    return
  soup = BeautifulSoup(response.content, 'html.parser')

  try:
    table = soup.find('table', {'id': table_id})
  except:
    print("Can't find table")
    return
  try:
    rows = table.find_all('tr')
  except:
    print("Table is empty")
    print("url:", url)
    print("table_id:", table_id, type(table_id))
    return

  data = []

  for row in rows:
    d = {}
    row_headers = row.find_all('th')
    if row_headers:
      for header in row_headers:
        if header.get('data-stat') == 'date': # Include date column
          date = header.get_text(strip=True)
          d['date'] = date
    # Get data from each row
    cells = row.find_all('td')
    if cells:
      for cell in cells:
        d[cell.get('data-stat')] = cell.get_text(strip=True)
      data.append(d)

  df = pd.DataFrame(data)
  df = df.drop(columns=['match_report']) # Irrelevant column
  return df


In [4]:
# Define function to get a dataframe that combines data from multiple seasons
def get_grouped_tables(url_template, season_range, matchlog, tabs, common_keys, add_season=True):
  table_id = "matchlogs_" + matchlog

  dfs = []
  for year in range(season_range[0], season_range[1]+1):
    season = str(year) + '-' + str(year+1)
    tmp_df = pd.DataFrame()
    for tab in tabs:
      url = url_template.format(season, tab) # Completes the url for a given season and a given tab
      df = get_table(url, table_id)
      print(".", end="") # To show progress
      if tmp_df.empty:
        tmp_df = df
      else:
        tmp_df = pd.merge(tmp_df, df, on=common_keys, how='outer')

      time.sleep(2) # Sleep for 2 seconds before making the next request (to prevent getting blocked by website)

    if add_season:
      tmp_df['season'] = year # Eg. 2020/21 season would be labelled 2020

    print(season, "SEASON COMPLETE")
    dfs.append(tmp_df)
    time.sleep(5) # Sleep for 5 seconds before next loop (to prevent getting blocked by website)

  final_df = pd.concat(dfs, ignore_index=True)
  final_df = final_df[final_df['date'] != ""] # drop rows that don't contain a date (some rows contain season totals, these should be removed)
  return final_df


In [5]:
url_template = "https://fbref.com/en/squads/18bb7c10/{}/matchlogs/c9/{}/Arsenal-Match-Logs-Premier-League"
season_range = [2020, 2023] # All seasons from 2020/21 to 2023/24
tabs = ['passing', 'shooting', 'passing_types', 'gca', 'defense', 'possession', 'misc'] # Different 'tabs' on the fbref website
common_keys = ['date', 'start_time', 'round', 'dayofweek', 'venue', 'result', 'goals_for', 'goals_against', 'opponent'] # These categories are contained in the tables in each tab, so can be used to merge dataframes reliably

combined_df = get_grouped_tables(url_template, season_range, 'against', tabs, common_keys)
display(combined_df)

.......2020-2021 SEASON COMPLETE
.......2021-2022 SEASON COMPLETE
.......2022-2023 SEASON COMPLETE
.......2023-2024 SEASON COMPLETE


,date,start_time,round,dayofweek,venue,result,goals_for,goals_against,opponent,passes_completed_x,...,interceptions_y,tackles_won_y,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,season
1,2020-09-12,12:30,Matchweek 1,Sat,Home,L,0,3,Fulham,485,...,10,7,0,0,0,46,8,5,61.5,2020
2,2020-09-19,20:00,Matchweek 2,Sat,Away,L,1,2,West Ham,340,...,10,5,0,0,0,34,14,16,46.7,2020
3,2020-09-28,20:00,Matchweek 3,Mon,Home,W,3,1,Liverpool,712,...,6,6,0,0,0,51,6,11,35.3,2020
4,2020-10-04,14:00,Matchweek 4,Sun,Away,L,1,2,Sheffield Utd,313,...,5,4,0,0,0,41,14,17,45.2,2020
5,2020-10-17,17:30,Matchweek 5,Sat,Home,W,1,0,Manchester City,604,...,8,5,0,0,0,47,15,12,55.6,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2024-04-23,20:00,Matchweek 29,Tue,Away,L,0,5,Chelsea,567,...,7,5,0,0,0,53,11,12,47.8,2023
152,2024-04-28,14:00,Matchweek 35,Sun,Home,L,2,3,Tottenham,525,...,8,12,1,0,1,45,9,13,40.9,2023
153,2024-05-04,12:30,Matchweek 36,Sat,Away,L,0,3,Bournemouth,314,...,10,9,0,1,0,40,19,15,55.9,2023
154,2024-05-12,16:30,Matchweek 37,Sun,Home,L,0,1,Manchester Utd,458,...,5,12,0,0,0,45,13,10,56.5,2023


In [6]:
combined_df_copy = combined_df.copy() # To ensure that we don't have to re-run the previous code if a mistake is made after this point

# Define function to convert to values to numeric where possible
def safe_to_numeric(x):
    if isinstance(x, str):
        try:
            return pd.to_numeric(x, errors='raise')
        except ValueError:
            return x
    return x

combined_df_copy = combined_df_copy.applymap(safe_to_numeric)

<ipython-input-6-9758bdac0e2d>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined_df_copy = combined_df_copy.applymap(safe_to_numeric)


## Display data

In [7]:
combined_df_copy

,date,start_time,round,dayofweek,venue,result,goals_for,goals_against,opponent,passes_completed_x,...,interceptions_y,tackles_won_y,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,season
1,2020-09-12,12:30,Matchweek 1,Sat,Home,L,0,3,Fulham,485,...,10,7,0,0,0,46,8,5,61.5,2020
2,2020-09-19,20:00,Matchweek 2,Sat,Away,L,1,2,West Ham,340,...,10,5,0,0,0,34,14,16,46.7,2020
3,2020-09-28,20:00,Matchweek 3,Mon,Home,W,3,1,Liverpool,712,...,6,6,0,0,0,51,6,11,35.3,2020
4,2020-10-04,14:00,Matchweek 4,Sun,Away,L,1,2,Sheffield Utd,313,...,5,4,0,0,0,41,14,17,45.2,2020
5,2020-10-17,17:30,Matchweek 5,Sat,Home,W,1,0,Manchester City,604,...,8,5,0,0,0,47,15,12,55.6,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2024-04-23,20:00,Matchweek 29,Tue,Away,L,0,5,Chelsea,567,...,7,5,0,0,0,53,11,12,47.8,2023
152,2024-04-28,14:00,Matchweek 35,Sun,Home,L,2,3,Tottenham,525,...,8,12,1,0,1,45,9,13,40.9,2023
153,2024-05-04,12:30,Matchweek 36,Sat,Away,L,0,3,Bournemouth,314,...,10,9,0,1,0,40,19,15,55.9,2023
154,2024-05-12,16:30,Matchweek 37,Sun,Home,L,0,1,Manchester Utd,458,...,5,12,0,0,0,45,13,10,56.5,2023


## Save to .csv

In [8]:
combined_df_copy.to_csv("arsenal_opponent_data.csv", index=False)